In [49]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()
from mysql_conn import password

import os

## Top 200 Daily

In [20]:
top_200_daily = pd.read_csv('spotify-chart/top_200_daily.csv')

top_200_daily.drop(columns = 'Unnamed: 0', inplace = True)
# top_200_weekly.rename(columns = {'Unnamed: 0':'index'}, inplace = True)
top_200_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

top_200_daily['ID'] = top_200_daily['URL'].str.split('/', expand = True)[4]

# top 200 db export
top_200_db = top_200_daily[['Position', 'date', 'ID']]
top_200_db.head()

,Position,date,ID
0,1,2017-01-01,4Km5HrUvYTaSUfiSGPJeQR
1,2,2017-01-01,343YBumqHu19cGoGARUTsd
2,3,2017-01-01,5aAx2yezTd8zXrkmtKl66Z
3,4,2017-01-01,7BKLCZ1jbUBVqRi2FVlTVw
4,5,2017-01-01,6fujklziTHa8uoM5OQSfIo


## Viral 50 Daily

In [22]:
viral_50_daily = pd.read_csv('spotify-chart/viral_50_daily.csv')

viral_50_daily.drop(columns = 'Unnamed: 0', inplace = True)
# top_200_weekly.rename(columns = {'Unnamed: 0':'index'}, inplace = True)
viral_50_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

viral_50_daily['ID'] = viral_50_daily['URL'].str.split('/', expand = True)[4]

# top 200 db export
viral_50_db = viral_50_daily[['Position', 'date', 'ID']]
viral_50_db.head()

,Position,date,ID
0,1,2017-01-01,1Th5b8AsauED9F7pGv8Yxi
1,2,2017-01-01,1D6nV9TPfMnWm7UdVsDVfI
2,3,2017-01-01,6F7CjVIrwk7PddQUsoPmoo
3,4,2017-01-01,0HEmnAUT8PHznIAAmVXqFJ
4,5,2017-01-01,0L0T4tMAaGqLgIVj1MOj9t


## Unique Songs

In [28]:
unique_songs = viral_50_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]
unique_songs.head()

,ID,Track,Artist
0,1Th5b8AsauED9F7pGv8Yxi,Fill Me In - AFTR:HRS Mixcut,Loud Luxury
1,1D6nV9TPfMnWm7UdVsDVfI,Freedom! '90,George Michael
2,6F7CjVIrwk7PddQUsoPmoo,You,Manwolves
3,0HEmnAUT8PHznIAAmVXqFJ,Faith - Remastered,George Michael
4,0L0T4tMAaGqLgIVj1MOj9t,Father Figure - Remastered,George Michael


## Features

# Export Song Tables to SQL

In [29]:
connection_string = (f"root:{password}@localhost")

engine = create_engine(f"mysql://{connection_string}")
engine.execute("DROP DATABASE IF EXISTS spot_db")
engine.execute("CREATE DATABASE spot_db")
engine.execute("ALTER DATABASE spot_db CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")
# engine.execute("ALTER TABLE indeed_resumes CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
engine = create_engine(f"mysql://{connection_string}/spot_db?charset=utf8mb4")

In [30]:
# Top 200 Daily
engine.execute("USE spot_db")
top_200_db.to_sql(
    name = 'top_200_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `top_200_daily` ADD PRIMARY KEY (`Index`);')

In [31]:
# Viral 50 Daily
engine.execute("USE spot_db")
viral_50_db.to_sql(
    name = 'viral_50_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `viral_50_daily` ADD PRIMARY KEY (`Index`);')

In [60]:
# Unique Songs
engine.execute("USE spot_db")
unique_songs.to_sql(
    name = 'songs', con = engine,
    if_exists = 'replace', 
    dtype={'ID': types.VARCHAR(255)})
with engine.connect() as con:
    con.execute('ALTER TABLE `songs` ADD PRIMARY KEY (`Id`);')